# 🎯 Fabric Spark Advisor — Quick Start

**Lightweight notebook for fast Spark optimization queries**

This is a simplified version optimized for Microsoft Fabric notebooks.
For full interactive UI, use `FabricSparkAdvisor_Interactive.ipynb`.

---

## 1️⃣ Setup

In [ ]:
# Import dependencies (most are pre-installed in Fabric)
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from agent.orchestrator import SparkAdvisorOrchestrator
from mcp_server.kusto_client import KustoClient
import uuid
import asyncio

print("✅ Imports successful!")

In [ ]:
# Initialize Spark Advisor
kusto_client = KustoClient.from_env()
orchestrator = SparkAdvisorOrchestrator.from_env()
session_id = str(uuid.uuid4())

print(f"✅ Spark Advisor initialized!")
print(f"Session ID: {session_id[:8]}...")

## 2️⃣ Quick Queries

In [ ]:
# Helper function for async calls
def ask(query: str):
    """Ask a question to the Spark Advisor"""
    loop = asyncio.get_event_loop()
    response = loop.run_until_complete(orchestrator.chat(query, session_id=session_id))
    print(response)
    return response

print("✅ Helper function ready. Use ask('your query') to get answers.")

### Example 1: Show Top 5 Slowest Applications

In [ ]:
ask("show top 5 slowest applications")

### Example 2: Analyze Specific Application

In [ ]:
# Replace with your application ID
app_id = "application_1771441543262_0001"
ask(f"analyze {app_id}")

### Example 3: Find Streaming Jobs

In [ ]:
ask("show streaming applications")

### Example 4: Show Bad Practice Apps

In [ ]:
ask("show bad practice applications")

### Example 5: Ask About Best Practices

In [ ]:
ask("what is VOrder and how does it improve performance?")

## 3️⃣ Direct Kusto Queries

In [ ]:
# Run custom KQL query
query = """
sparklens_metrics
| where metric == "Total Executor Time (sec)"
| top 10 by value desc
| project application_id, value, timestamp
"""

results = kusto_client.execute_query(query)

print("📊 Top 10 Applications by Executor Time:\n")
for i, row in enumerate(results, 1):
    app_id = row.get('application_id', 'N/A')
    value = row.get('value', 0)
    timestamp = row.get('timestamp', 'N/A')
    print(f"{i}. {app_id}: {value:.1f}s ({timestamp})")

## 4️⃣ Get Recommendations Directly

In [ ]:
# Get SparkLens recommendations for specific app
app_id = "application_1771441543262_0001"  # Replace with your app ID

spark_recs = orchestrator.get_spark_recommendations(app_id)
fabric_recs = orchestrator.get_fabric_recommendations(app_id)

print(f"🔍 Analysis for {app_id}\n")
print(f"{'='*60}\n")

if spark_recs:
    print(f"📌 SparkLens Recommendations ({len(spark_recs)}):")
    for i, rec in enumerate(spark_recs, 1):
        severity = rec.get('severity', 'UNKNOWN')
        message = rec.get('message', 'No message')
        print(f"\n{i}. [{severity}] {message}")
else:
    print("📌 SparkLens Recommendations: No data found")

print(f"\n{'='*60}\n")

if fabric_recs:
    print(f"📌 Fabric Recommendations ({len(fabric_recs)}):")
    for i, rec in enumerate(fabric_recs, 1):
        severity = rec.get('severity', 'UNKNOWN')
        message = rec.get('message', 'No message')
        print(f"\n{i}. [{severity}] {message}")
else:
    print("📌 Fabric Recommendations: No data found")

## 5️⃣ Provide Feedback

In [ ]:
# Save feedback on the last query
def save_feedback(feedback_type: str, comment: str = ""):
    """
    Save feedback to Kusto
    
    Args:
        feedback_type: "HELPFUL", "NOT_HELPFUL", or "PARTIAL"
        comment: Optional comment explaining your feedback
    """
    try:
        success = kusto_client.insert_feedback(
            session_id=session_id,
            application_id=app_id,
            query_text=f"analyze {app_id}",
            query_intent="analyze_app",
            actual_result_generated="See recommendations above",
            feedback_type=feedback_type,
            feedback_comment=comment,
            recommendation_count=len(spark_recs or []) + len(fabric_recs or []),
            kusto_recommendation_count=len(spark_recs or []),
            rag_recommendation_count=0,
            llm_recommendation_count=0
        )
        
        if success:
            print(f"✅ Thank you! Your {feedback_type} feedback has been saved.")
        else:
            print("❌ Failed to save feedback.")
    except Exception as e:
        print(f"❌ Error saving feedback: {e}")

# Example usage:
# save_feedback("HELPFUL", "The executor efficiency recommendation was spot on!")
# save_feedback("NOT_HELPFUL", "Recommendations didn't apply to my batch job")
# save_feedback("PARTIAL", "Some recommendations were useful, others not")

print("✅ Feedback function ready. Use save_feedback('HELPFUL', 'your comment') to provide feedback.")

## 6️⃣ Common Queries Cheat Sheet

In [ ]:
# Copy-paste these queries to get started quickly:

queries = {
    "Top 5 slowest apps": "show top 5 slowest applications",
    "Top 10 slowest apps": "show top 10 applications by time",
    "Streaming jobs": "show streaming applications",
    "Bad practice apps": "show bad practice applications",
    "Best practice apps": "show healthy applications",
    "Analyze specific app": "analyze application_1771441543262_0001",
    "VOrder info": "what is VOrder?",
    "Shuffle optimization": "how to optimize shuffle operations?",
    "Memory tuning": "how to tune memory settings?",
    "Recent apps": "show recent applications"
}

print("📋 Common Queries:\n")
for name, query in queries.items():
    print(f"  • {name}:")
    print(f"    ask('{query}')\n")

---

## 🚀 Next Steps

1. **Run your own queries** using the `ask()` function
2. **Provide feedback** to help improve recommendations
3. **Explore documentation** for advanced features:
   - [ARCHITECTURE.md](../ARCHITECTURE.md)
   - [TOOL_REFERENCE.md](../components/TOOL_REFERENCE.md)
   - [Feedback Learning Strategy](../mcp_server/feedback_learning_strategy.md)

---

**Built with ❤️ by the Fabric Spark Advisor Team**